## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier

In [2]:
# 讀取數字資料集
digits = datasets.load_digits()
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(digits.data, digits.target, test_size=0.25, random_state=42)

# 建立模型
clf = RandomForestClassifier(random_state=7)

In [3]:
# 先看看使用預設參數得到的結果
clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
print(metrics.accuracy_score(y_test, y_pred))

0.98


In [4]:
# 設定要訓練的超參數組合
n_estimators = [100, 200, 300]
max_depth = [1, 3, 5, 7]
max_features = [None, 'sqrt', 'log2'] 

hyperparameters = {
    'n_estimators': n_estimators,
    'max_depth' : max_depth,   
    'max_features' : max_features
}

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid=hyperparameters, scoring="accuracy", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

# 預設會跑 5-fold cross-validadtion，總共 36 種參數組合，總共要 train 180 次模型

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.3s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:  1.2min finished


In [5]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: 0.963621 using {'max_depth': 7, 'max_features': 'log2', 'n_estimators': 200}


In [6]:
# 使用最佳參數重新建立模型
clf_bestparam = RandomForestClassifier(max_depth=grid_result.best_params_['max_depth'],
                                        n_estimators=grid_result.best_params_['n_estimators'],
                                       max_features=grid_result.best_params_['max_features'])

# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [7]:
print(metrics.accuracy_score(y_test, y_pred))

0.9688888888888889


In [9]:
grid_search.cv_results_

{'mean_fit_time': array([0.49949236, 1.11421318, 1.78205628, 0.33960958, 0.96203399,
        1.20513983, 0.39742503, 0.87946005, 1.46671081, 0.93173571,
        1.81575809, 2.79182935, 0.49107304, 0.90050898, 1.69373574,
        0.50588274, 0.9217392 , 1.79162531, 1.65313649, 2.78187451,
        4.04493613, 0.51159396, 0.99529862, 1.72862411, 0.67540359,
        1.08159165, 1.5289784 , 1.89019499, 3.78338795, 6.35141129,
        0.9351965 , 1.35997486, 1.72059641, 0.50698576, 1.00659819,
        1.42320757]),
 'std_fit_time': array([0.03199287, 0.06277486, 0.13238693, 0.01051492, 0.14883413,
        0.02396304, 0.04880116, 0.10492911, 0.1714643 , 0.02727899,
        0.0260116 , 0.08666718, 0.03338057, 0.02807517, 0.12780223,
        0.08739768, 0.07978181, 0.05761728, 0.04096626, 0.08940876,
        0.04897637, 0.01573902, 0.02683371, 0.12716853, 0.02757557,
        0.03215688, 0.03748327, 0.05465701, 0.16799485, 0.45727698,
        0.1118926 , 0.02235547, 0.05125145, 0.00486037, 0.014